In [1]:
# testing remaining recommender methods
import collections

import compress_pickle
import copy
import editdistance
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
FIGURE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/figure_files'
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [3]:
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')
BIOMD_634_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000634.xml')
E_COLI_PATH = os.path.join(cn.TEST_DIR, 'e_coli_core.xml')
ONE_SPEC_CAND = ('CHEBI:15414', 1.0)
ONE_SPEC_URL = 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A15414'
TWO_SPEC_CAND = ('CHEBI:15729', 1.0)
TWO_SPEC_URL = 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A15729'

ONE_REAC_CAND = ('RHEA:28827', 1.0)
ONE_REAC_URL = 'https://www.rhea-db.org/rhea/28827'

SPECIES_SAM = 'SAM'
SPECIES_SAM_NAME = 'S-adenosyl-L-methionine'
SPECIES_ORN = 'ORN'
SPECIES_ATP = 'ATP'
REACTION_ODC = 'ODC'
REACTION_SAMDC = 'SAMdc'
REACTION_SPMS = 'SpmS'
R_PFK = 'R_PFK'
R_PFL = 'R_PFL'
ECOLI_REACTIONS = [R_PFK, R_PFL]
ECOLI_ATP = 'M_atp_c'
ECOLI_RHEA = 'RHEA:12420'

ONE_CHEBI = 'CHEBI:15414'
ATP_CHEBI = 'CHEBI:30616'
FORMULA_ATP = 'C10N5O13P3'

RESULT_RECOM = cn.Recommendation('R_PFK', 0.817,
                                 [('RHEA:12420', 0.6), ('RHEA:13377', 0.6)],
                                 ['https://www.rhea-db.org/rhea/12420', 'https://www.rhea-db.org/rhea/13377'],
                                 ['tagatose-6-phosphate kinase activity', 'phosphoglucokinase activity'])

In [4]:
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)

In [10]:
result_df = recom.getDataFrameFromRecommendation(RESULT_RECOM)
result_df

,annotation,match score,label
R_PFK (cred. 0.817),,,
1,RHEA:12420,0.6,tagatose-6-phosphate kinase activity
2,RHEA:13377,0.6,phosphoglucokinase activity


In [13]:
res1 = recom.filterDataFrameByThreshold(result_df, min_score=0.6)
res1

,annotation,match score,label
R_PFK (cred. 0.817),,,
1,RHEA:12420,0.6,tagatose-6-phosphate kinase activity
2,RHEA:13377,0.6,phosphoglucokinase activity


In [14]:
res2 = recom.filterDataFrameByThreshold(result_df, min_score=0.7)
res2

,annotation,match score,label
R_PFK (cred. 0.817),,,


In [24]:
recom.getReactionIDs()

['ODC',
 'SAMdc',
 'SSAT_for_S',
 'SSAT_for_D',
 'PAO_for_aD',
 'PAO_for_aS',
 'SpdS',
 'SpmS',
 'MAT',
 'VCoA',
 'VacCoA',
 'P_efflux',
 'aD_efflux']

In [26]:
recom.recommendReaction(ids=['ODC'], min_score=0.6)

                                        ODC (credibility score: 0.815)                                       
+----+--------------+---------------+-----------------------------------------------------------------------+
|    | annotation   |   match score | label                                                                 |
+====+==============+===============+=======================================================================+
|  1 | RHEA:28827   |         1.000 | L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out) |
+----+--------------+---------------+-----------------------------------------------------------------------+



In [29]:
res = recom.getReactionListRecommendation(pred_ids=['ODC'],
                                         get_df=True)
res_dict = {val:res[idx] for idx, val in enumerate(['ODC'])}
for k in res_dict.keys():
  filt_df = recom.filterDataFrameByThreshold(res_dict[k], 0.6)
  s = recom.getMarkdownFromRecommendation(filt_df)+"\n"

In [30]:
s

'                                        ODC (credibility score: 0.815)                                       \n+----+--------------+---------------+-----------------------------------------------------------------------+\n|    | annotation   |   match score | label                                                                 |\n+====+==============+===============+=======================================================================+\n|  1 | RHEA:28827   |         1.000 | L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out) |\n+----+--------------+---------------+-----------------------------------------------------------------------+\n'